In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import requests
import sys
import re
import time
import pyperclip
import os
import osascript
from subprocess import call
import json
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
import traceback
import random
import string
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

# Connect to a given url with ChromeDriver using a proxy from Tor
def chrome_with_proxy(url):

    USER_AGENT_LIST = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
                    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
                    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                ]

    # Killing previous and starting new Chrome session with Tor proxy, going to register website
    code,out,err = osascript.run('quit app "Tor Browser"')
    time.sleep(2)
    os.system("open /Applications/TorBrowser.app")
    time.sleep(3)

    # Running Chrome with proxy
    print('Running Chrome with proxy...')
    PROXY = "socks5://127.0.0.1:9150"
    
    options = webdriver.ChromeOptions()
    USER_AGENT = random.choice(USER_AGENT_LIST)
    options.add_argument('--user-agent=%s' % USER_AGENT)
    options.add_argument('--proxy-server=%s' % PROXY)
    driver = uc.Chrome(version_main=105, options=options)
    wait = WebDriverWait(driver, 10)
    driver.maximize_window()

    driver.get("https://api.ipify.org?format=json")
    ip_res = json.loads(driver.find_element(By.TAG_NAME, "body").text)
    print('Current IP:', ip_res['ip'])
    print('User agent:', USER_AGENT)

    driver.get(url)
    return driver

def recaptcha_solver_api(driver):
    print('Running 2captcha solver...')
    wait = WebDriverWait(driver, 10)
    # 2captcha solver API
    captcha_in = 'http://2captcha.com/in.php'
    captcha_out = 'http://2captcha.com/res.php'
    try:
        # Get sitekey assigned from Google captcha to this website
        sitekey = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.g-recaptcha"))).get_attribute('data-sitekey')
    except:
        print('ERR! Recaptcha sitekey not found')
    # Send captcha to be solved
    resp = requests.post(captcha_in, data={
        'key': 'a6ac106169706a05bdee349ffa74aba4',
        'method': 'userrecaptcha',
        'googlekey': sitekey,
        'pageurl': driver.current_url})
    captcha_id = resp.text.split('|')[1]
    print('Captcha id', captcha_id, 'sent to be solved, waiting 25 seconds to get response...')
    time.sleep(15)
    r = ''
    while (r == '' or r == 'CAPCHA_NOT_READY'):
        if (r == 'CAPCHA_NOT_READY'):
            print('Captcha not ready, trying again in 5 seconds...')
        time.sleep(5)
        resp_get = requests.get(captcha_out, params={
            'action': 'get',
            'key': 'a6ac106169706a05bdee349ffa74aba4',
            'id': captcha_id
            })
        r = resp_get.text
    if('OK' in r):
        t = 'OK'
        solution = r.split('|')[1]
    else:
        t = r
    print('Captcha response received:', t)
    # Apply captcha solution and enter website
    try:
        script = 'document.getElementById("g-recaptcha-response").innerHTML="{solution}";'.format(solution=solution)
        driver.execute_script(script)
        return 'success'
    except:
        print("ERR! Can't apply captcha solution")

def get_search_credits(driver):
    # Get keysearch credits (free searches)
    wait = WebDriverWait(driver, 10)
    credits = -1
    while(credits < 0):
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[1]/nav/div[2]/ul[2]/li/a"))).click()
        print('Getting free credits left, make sure the browser window is full-screen')
        time.sleep(10)
        credits = int(wait.until(EC.element_to_be_clickable((By.ID, "layout_credit"))).text[-1])
        if(credits >= 0):
            print(credits, 'free credits left')
            return credits

def keysearch_login(url):
    # Connect to main keysearch.co site with Selenium ChromeDriver
    driver = uc.Chrome(version_main=105)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    # If not logged in, attempt logging in with last used credentials
    if('research' not in driver.current_url):
        print('Not logged in, attempting to use last saved credentials...')
    driver.get('https://www.keysearch.co/user/login')
    wait.until(EC.element_to_be_clickable((By.ID, "identity"))).send_keys('1vc40exgwk@dcctb.com')
    wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys('123456')
    # Solve captcha
    recaptcha_solver_api(driver)
    # Submit login form
    wait.until(EC.element_to_be_clickable((By.ID, "submit"))).click()
    return driver

def locate_recaptcha_audio(driver):
        wait = WebDriverWait(driver, 10)
        time.sleep(3)
        try:
            frame = wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='reCAPTCHA']")))
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()
        except:
            print("ERR! Can't find recaptcha iframe")
            return 'err'
        try:
            driver.switch_to.default_content()
            # wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='recaptcha challenge']")))
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.xpath("//div[contains(text(),"+"recaptcha challenge"+")]"))))
            wait.until(EC.element_to_be_clickable((By.ID, "recaptcha-audio-button"))).click()
        except:
            print("ERR! Can't locate recaptcha audio")
            return 'err'
        return 'success'
        # # switch to recaptcha audio control frame
        # time.sleep(3)
        # driver.switch_to.default_content()
        # driver.switch_to.frame(recaptcha_challenge_frame)
        # # click on audio challenge
        # time.sleep(10)
        # driver.find_element_by_id("recaptcha-audio-button").click()

def input_keywords():
    keywords = []
    n_keywords = int(input("Enter number of queries to search (type 0 to search 'best pizza in Copenhagen'): "))
    if(n_keywords == 0):
        return ['best pizza in Copenhagen']
    else:
        for i in range(0, n_keywords):
            keyword = input("Enter next keyword")
            keywords.append(keyword)
    print('Getting keywords:', keywords)
    return keywords

def scrape(driver, keyword):
    wait = WebDriverWait(driver, 10)
    # Search for keyword
    print('Scraping next keyword:', keyword)
    wait.until(EC.element_to_be_clickable((By.ID, "keyword"))).clear()
    wait.until(EC.element_to_be_clickable((By.ID, "keyword"))).send_keys(keyword)
    wait.until(EC.element_to_be_clickable((By.ID, "search_button"))).click()
    print('Scraping, please wait...')
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    div=soup.find('table', id='url-tab-data')
    table=pd.read_html(str(div))
    print(table)
    table.to_csv()


def connect_via_proxy():
    failure_reason = 'x'
    while (len(failure_reason) != 0):

        # Killing previous and starting new Chrome session with Tor proxy, going to register website
        code,out,err = osascript.run('quit app "Tor Browser"')
        time.sleep(3)
        os.system("open /Applications/TorBrowser.app")
        time.sleep(3)

        # Running Chrome with proxy
        print('Running Chrome with proxy...')
        PROXY = "socks5://127.0.0.1:9150"
        options = webdriver.ChromeOptions()
        options.add_argument('--proxy-server=%s' % PROXY)
        driver2 = uc.Chrome(options=options)
        driver2.maximize_window()

        driver2.get("https://api.ipify.org?format=json")
        ip_res = json.loads(driver2.find_element(By.TAG_NAME, "body").text)
        print('Current IP:', ip_res['ip'])

        driver2.get("https://www.keysearch.co/user/register?plan=25")
        time.sleep(3)
       

def is_ip_blocked(driver):
    ip_blocked = driver.find_elements(By.XPATH, "//*[contains(text(),'" + 'Checking if the site connection is secure' + "')]")
    if(len(ip_blocked)):
        return True
    else:
        return False

def is_ip_taken(driver):
    ip_taken = driver.find_elements(By.XPATH, "//*[contains(text(),'" + 'Account with This IP is already exits' + "')]")
    if(len(ip_taken)):
        return True
    else:
        return False

def is_registration_locked(driver):
    # no_free_accounts = driver.find_elements(By.CSS_SELECTOR, "div.alert_dismissable")
    no_free_accounts = driver.find_elements(By.XPATH, "//*[contains(text(),'" + 'overwhleming demand' + "')]")
    if(len(no_free_accounts)):
        return True
    else:
        return False

def is_registration_successful(driver):
    success = driver.find_elements(By.XPATH, "//*[contains(text(),'" + 'You can check your email address for verification.' + "')]")
    success += driver.find_elements(By.XPATH, "//*[contains(text(),'" + 'confirmation' + "')]")
    if(len(success)):
        return True
    else:
        return False

def generate_username():
    name = string.ascii_lowercase + string.digits
    username = ''.join(random.choice(name) for i in range(10))
    return username

def new_mail():
    def extract():
        get_username = re.search(r'login=(.*)&',new_mail).group(1)
        get_domain = re.search(r'domain=(.*)', new_mail).group(1)
        return [get_username, get_domain]

    API = 'https://www.1secmail.com/api/v1/'
    domain_list = ['1secmail.com', '1secmail.net', '1secmail.org']
    domain = random.choice(domain_list)
    new_mail = f"{API}?login={generate_username()}&domain={domain}"
    req_mail = requests.get(new_mail)
    mail = f"{extract()[0]}@{extract()[1]}"
    return mail

def check_mails(email):
    API = 'https://www.1secmail.com/api/v1/'
    confirmation_link = []
    while (len(confirmation_link) == 0) :
        time.sleep(5)

        reqLink = f'{API}?action=getMessages&login={email.split("@")[0]}&domain={email.split("@")[1]}'
        req = requests.get(reqLink).json()
        length = len(req)
        if length == 0:
            print("Waiting for confirmation link. Mailbox is refreshed automatically every 5 seconds...")
        else:
            idList = []
            for i in req:
                for k,v in i.items():
                    if k == 'id':
                        mailId = v
                        idList.append(mailId)

            for i in idList:
                msgRead = f'{API}?action=readMessage&login={email.split("@")[0]}&domain={email.split("@")[1]}&id={i}'
                req = requests.get(msgRead).json()
                soup = BeautifulSoup(req['body'], 'html.parser')
                confirmation_link = soup.find_all("a")
                print('Confirmation link received')
                
    return confirmation_link[0].text

def confirm_account(email, driver):
    confirmation_link = check_mails(email)
    driver.get(confirmation_link)

def add_account_to_list(email):
    with open('accounts.txt', 'a') as f:
        f.write(email + ';' + datetime.now() + '\n')

def get_previous_accounts():
    with open('accounts.txt', 'r') as f:
        accounts = f.readlines()
        emails = [line.split(";")[0] for line in accounts]
        dates = [line.split(";")[1] for line in accounts]
    return zip(emails, dates)
        
        

def keysearch_register(driver):
    wait = WebDriverWait(driver, 10)
    print('Registering new account...')
    
    # Create new email address
    call(["python", "mailSwipe.py"])
    time.sleep(2)
    new_mail = pyperclip.paste()
    time.sleep(2)
    print('New temp email address is:', new_mail)

    wait.until(EC.element_to_be_clickable((By.ID, "id_email"))).send_keys(new_mail) 
    wait.until(EC.element_to_be_clickable((By.ID, "id_password"))).send_keys('123456')
    wait.until(EC.element_to_be_clickable((By.ID, "id_passwordVerify"))).send_keys('123456')
    
    # Tab and space to check Terms and Conditions
    actions = ActionChains(driver)
    actions.send_keys(Keys.TAB * 5)
    actions.send_keys(Keys.SPACE)
    actions.perform()

    # Solve register captcha
    recaptcha_solver_api(driver)
    wait.until(EC.element_to_be_clickable((By.ID, "id_btn"))).click()
    time.sleep(3)

# ----------------------------------------------------------------

url = 'https://www.keysearch.co/'

# 0 Ask user for keywords before running - INPUT 0 FOR THE DEFAULT FUNCTIONALITY
# keywords = input_keywords()
keywords = ['best pizza in Copenhagen']
if(len(keywords)==0):
    keywords = ['best pizza in Copenhagen']
# 1 Get keysearch.co - if not logged in, try last credentials
driver = keysearch_login(url)

# 1.1 Last credentials from txt that were unused for the past 24 hours
accounts = get_previous_accounts()
refreshed_accounts = []
for acc in accounts:
    if(datetime.now() - accounts[acc] > timedelta(days=1)):
        refreshed_accounts += acc

# 2 Iterate over all keywords
for keyword in keywords:
    # 3 Check credits left every time (some keywords that were searched before do not take a credit)
    # credits = get_search_credits(driver)
    credits = 1
    # 4 If credits > 0 proceed to scraping
    if (credits > 0):
        scrape(driver, keyword)
    # 5 If credits = 0, log out
    if (credits == 0):
        print('No credits left, using different credentials...')
        driver.get('https://www.keysearch.co/user/logout')
        time.sleep(3)

    if (len(refreshed_accounts)):
        # 5.1 If any accounts in txt are refreshed, use them
        driver = keysearch_login(url)

    else:
        # 5.2 ... if no accounts available, create new account
        register_url = 'https://www.keysearch.co/user/register?plan=25'
        error = 'x'
        while(len(error)):
            try:
                driver2.close()
            except:
                pass

            # Access register URL through proxy (they check for IPs)
            driver2 = chrome_with_proxy(register_url)
            wait2 = WebDriverWait(driver2, 10)

            try:
                # Check if accessed register website
                wait2.until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'" + 'Create Your Account!' + "')]")))
            except:
                # If did not access register website, check if it's a Cloudfare block
                if (is_ip_blocked(driver2)):
                    print("This IP may be blocked by Cloudfare, trying with different IP...")
                    continue
            else:
                # Proceed to registering
                new_mail = keysearch_register(driver2)
                time.sleep(3)

                # Check for errors, and lastly check if successful
                if (is_ip_taken(driver2)):
                    print('This IP is already taken, trying with different IP...')
                    continue
                elif (is_registration_locked(driver2)):
                    print('Cannot register new account right now, try again later.')
                    error = ''
                elif (is_registration_successful(driver2)):
                    print('Registration successfull')
                    confirm_account(new_mail, driver2)
                    add_account_to_list(new_mail)
                    error = ''


Scraping next keyword: what


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   3458d6c583954a6e_chromedriver       0x00000001029e3788 3458d6c583954a6e_chromedriver + 4515720
1   3458d6c583954a6e_chromedriver       0x00000001029679d3 3458d6c583954a6e_chromedriver + 4008403
2   3458d6c583954a6e_chromedriver       0x00000001025fa12a 3458d6c583954a6e_chromedriver + 413994
3   3458d6c583954a6e_chromedriver       0x00000001025dbad4 3458d6c583954a6e_chromedriver + 289492
4   3458d6c583954a6e_chromedriver       0x00000001025dfd4d 3458d6c583954a6e_chromedriver + 306509
5   3458d6c583954a6e_chromedriver       0x00000001025e4e7c 3458d6c583954a6e_chromedriver + 327292
6   3458d6c583954a6e_chromedriver       0x00000001025fb6b2 3458d6c583954a6e_chromedriver + 419506
7   3458d6c583954a6e_chromedriver       0x0000000102661221 3458d6c583954a6e_chromedriver + 836129
8   3458d6c583954a6e_chromedriver       0x000000010264e603 3458d6c583954a6e_chromedriver